## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import tensorflow as tf

# Install and import keras_tuner
!pip install keras_tuner
import keras_tuner as kt

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

Using TensorFlow backend


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_value_counts = application_df["APPLICATION_TYPE"].value_counts()

In [5]:
# Choose a modified cutoff value for application types to be replaced
# Use the variable name `application_types_to_replace`
application_types_to_replace = app_value_counts[app_value_counts < 1000].index

# Replace in DataFrame
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_value_counts = application_df['CLASSIFICATION'].value_counts()

# Choose a modified cutoff value (1500) for classifications to be replaced
# Use the variable name `classifications_to_replace`
classifications_to_replace = class_value_counts[class_value_counts < 1500].index

# Replace in DataFrame
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
app_converted_df = pd.get_dummies(application_df)
app_converted_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [8]:
# Split our preprocessed data into our features and target arrays
y = app_converted_df["IS_SUCCESSFUL"].values
X = app_converted_df.drop(["IS_SUCCESSFUL"], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

<ipython-input-8-ab27e893e1c8>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = app_converted_df.drop(["IS_SUCCESSFUL"], 1).values


In [9]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [10]:
# Define the model - deep neural net i.e., the number of input features and hidden nodes for each layer.

input_dim = len(X_train[0])

def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    # Optimization: Increased the min_value, max_value, and step value for more variety
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=10,
        max_value=90,
        step=5), activation=activation, input_dim=input_dim))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    # Optimization: Increased the min_value, max_value, and step value for more variety
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=5,
            max_value=30,
            step=5),
            activation=activation))

    # Add final layer with sigmoid activation function
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [11]:
# Set up the kerastuner then allow it to search for best hyperparameters
# Optimization: Increased number of epochs in tuner creation and search to potentially improve performance

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=30,
    hyperband_iterations=2)

tuner.search(X_train_scaled, y_train, epochs=30, validation_data=(X_test_scaled, y_test))

Trial 180 Complete [00h 01m 24s]
val_accuracy: 0.7269970774650574

Best val_accuracy So Far: 0.7292128205299377
Total elapsed time: 01h 11m 34s


In [12]:
# Get the hyperparameters of the best model
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 55,
 'num_layers': 5,
 'units_0': 20,
 'units_1': 20,
 'units_2': 10,
 'units_3': 15,
 'tuner/epochs': 30,
 'tuner/initial_epoch': 10,
 'tuner/bracket': 3,
 'tuner/round': 3,
 'units_4': 5,
 'tuner/trial_id': '0047',
 'units_5': 5}

In [13]:
# Get the best model
best_model = tuner.get_best_models(1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 55)                2255      
                                                                 
 dense_1 (Dense)             (None, 20)                1120      
                                                                 
 dense_2 (Dense)             (None, 20)                420       
                                                                 
 dense_3 (Dense)             (None, 10)                210       
                                                                 
 dense_4 (Dense)             (None, 15)                165       
                                                                 
 dense_5 (Dense)             (None, 5)                 80        
                                                                 
 dense_6 (Dense)             (None, 1)                 6

In [14]:
# Train the model
fit_model = best_model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5461 - accuracy: 0.7371
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5456 - accuracy: 0.7369
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5455 - accuracy: 0.7375
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5455 - accuracy: 0.7376
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accuracy: 0.7362
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5451 - accuracy: 0.7372
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5455 - accuracy: 0.7370
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5454 - accuracy: 0.7371
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5453 - accuracy: 0.7376
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5446 - accura

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = best_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5599 - accuracy: 0.7261 - 532ms/epoch - 2ms/step
Loss: 0.5598956942558289, Accuracy: 0.726064145565033


In [16]:
# Use the model to make predictions on test data
y_pred_prob = best_model.predict(X_test_scaled).flatten()
y_pred = [1 if prob >= 0.5 else 0 for prob in y_pred_prob]

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Generate a classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

268/268 [==============================] - 1s 2ms/step
Confusion Matrix:
[[2580 1423]
 [ 926 3646]]
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.64      0.69      4003
           1       0.72      0.80      0.76      4572

    accuracy                           0.73      8575
   macro avg       0.73      0.72      0.72      8575
weighted avg       0.73      0.73      0.72      8575



In [17]:
# Export our model to HDF5 file
best_model.save_weights('AlphabetSoupCharity_Optimization_3.hdf5')